In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from IPython.display import Image  
from sklearn import tree
import pydotplus
import cx_Oracle
import os
import pandas as pd 
import datetime

In [19]:
connection = cx_Oracle.connect('HCIN_GUPTAN1[AP_UWI]/aGqQ4jnX92NG@INCL02.IN.PROD/HWIN_USR_DEV.HOMECREDIT.IN')
cursor = connection.cursor()
os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\EWS")
qu1= """
 select * from TBD_EWS_FEATURES_DPD120
 """

df = pd.read_sql(qu1,connection)
df.to_pickle(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\EWS\segmentation_dump.pkl")
df.to_csv(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\EWS\segmentation_dump.csv")

In [23]:
df=df[(df["TIME_DECISION_DATE"]>=datetime.datetime(2018, 1, 1)) & 
              (df["TIME_DECISION_DATE"]<=datetime.datetime(2018, 12, 31))]

In [24]:
df.shape

(4856465, 44)

In [27]:
#Downsampling Code 
count_class_0, count_class_1 = df.DPD120.value_counts()
df_class_0 = df[df['DPD120'] == 0]
df_class_1 = df[df['DPD120'] == 1]
df_class_0_under = df_class_0.sample(count_class_1)
df_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_under.DPD120.value_counts())

df_under.DPD120.value_counts().plot(kind='bar', title='Count (target)');

Random under-sampling:
1    434008
0    434008
Name: DPD120, dtype: int64


In [28]:
df_under.to_pickle(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\EWS\segmentation_dump_undersample.pkl")
df_under.to_csv(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\EWS\segmentation_dump_undersample.csv")

In [58]:
# Load data
df = pd.read_csv(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\EWS\segmentation_dump_undersample.csv")


In [60]:
df.describe().transpose().to_csv(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\EWS\describe.csv")

In [61]:
df=df.select_dtypes(exclude=['object'])
#df =  df.drop(columns= ['SCORE_CRIF','SCORE_EQUIFAX','SCORE_EXPERIAN'])
df.dropna(inplace=True)
X = df.drop(columns= ['DPD120','SKP_CLIENT','Unnamed: 0','SKP_CREDIT_CASE'])
y = df["DPD120"]

In [62]:
df.shape

(242257, 28)

In [63]:
# Create decision tree classifer object
clf = DecisionTreeClassifier(random_state=0, max_depth=4)
# Train model
model = clf.fit(X, y)

In [69]:
# Create DOT data
dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=X.columns,  
                                class_names='FPD120')

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)  

# Show graph ---Not working due to graphViz
#Image(graph.create_png())

In [70]:
print(dot_data)
#copy this to editor window on http://www.webgraphviz.com/ to create chart

digraph Tree {
node [shape=box] ;
0 [label="RISK_SCORE <= 0.022\ngini = 0.491\nsamples = 242257\nvalue = [137595, 104662]\nclass = F"] ;
1 [label="SCORE_EXPERIAN <= 782.5\ngini = 0.44\nsamples = 131167\nvalue = [88378, 42789]\nclass = F"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="RISK_SCORE <= 0.007\ngini = 0.489\nsamples = 58707\nvalue = [33611, 25096]\nclass = F"] ;
1 -> 2 ;
3 [label="RISK_SCORE <= 0.004\ngini = 0.422\nsamples = 12411\nvalue = [8657, 3754]\nclass = F"] ;
2 -> 3 ;
4 [label="gini = 0.322\nsamples = 3439\nvalue = [2746, 693]\nclass = F"] ;
3 -> 4 ;
5 [label="gini = 0.45\nsamples = 8972\nvalue = [5911, 3061]\nclass = F"] ;
3 -> 5 ;
6 [label="SCORE_CRIF <= 712.5\ngini = 0.497\nsamples = 46296\nvalue = [24954, 21342]\nclass = F"] ;
2 -> 6 ;
7 [label="gini = 0.5\nsamples = 28864\nvalue = [13994, 14870]\nclass = P"] ;
6 -> 7 ;
8 [label="gini = 0.467\nsamples = 17432\nvalue = [10960, 6472]\nclass = F"] ;
6 -> 8 ;
9 [label="ACC_TERM <= 6.5\ngini

In [7]:
# Create PDF
graph.write_pdf("iris.pdf")

# Create PNG
graph.write_png("iris.png")